# Importing the dataset

In [2]:
!pip install pandas

     ---------------------------------------- 10.6/10.6 MB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 15.8/15.8 MB 4.4 MB/s eta 0:00:00
     -------------------------------------- 502.5/502.5 kB 3.9 MB/s eta 0:00:00
     -------------------------------------- 341.8/341.8 kB 4.3 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: C:\Users\USER\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [8]:
import requests 
import time
import json
import pandas as pd

## Get the api response

In [24]:
URL = "https://americas.api.riotgames.com"

def get_summoner_id(summoner_name, api_key):
    url = f'{URL}/lol/summoner/v4/summoners/by-name/{summoner_name}?api_key={api_key}'
    response = requests.get(url)
    time.sleep(1)
    return response.json()

def get_match_list(api_key, summoner_id, start=0, count=10):
    url = f'{URL}/lol/match/v5/matches/by-puuid/{summoner_id}/ids?start={start}&count={count}&api_key={api_key}'
    response = requests.get(url)
    time.sleep(1)
    return response.json()

def get_match_data(match_id, api_key):
    url = f'{URL}/lol/match/v5/matches/{match_id}?api_key={api_key}'
    response = requests.get(url)
    time.sleep(1)
    return response.json()

def extract_match_info(match_data):
    game_duration = match_data['info']['gameDuration']
    game_mode = match_data['info']['gameMode']
    participants = match_data['info']['participants']
    
    participants_df = pd.DataFrame()
    
    for participant in participants:
        participant_info = {
            'championName': participant['championName'],
            'kills': participant['kills'],
            'death': participant['deaths'],
            'assists': participant['assists'],
            'totalDamageToChampions': participant['totalDamageDealtToChampions'],
            'goldEarned': participant['goldEarned'],
            'visioScore': participant['visionScore'],
            'role': participant['individualPosition'],
            'teamId': participant['teamId']
        }
        participants_df = pd.concat([participants_df, pd.DataFrame([participant_info])], ignore_index=True)
      
        
    return game_duration, game_mode, participants_df

api_key = ''
summoner_id = 'YKfiig2deMN2x_h6ifhd6PhXDGCwhxlhxzqLv_I1Ca17pegJT3v39UIOF1lewJufghu5_pdRyH103g'


## Get the match ids

In [12]:
match_list = get_match_list(api_key, summoner_id, 0, 100)

for match in match_list:
    print(match)

BR1_2847053435
BR1_2843172863
BR1_2842671445
BR1_2839902431
BR1_2839881285
BR1_2781568879
BR1_2771129978
BR1_2771108401
BR1_2750887149
BR1_2694862769
BR1_2669047041
BR1_2665961872
BR1_2659616048
BR1_2656700458
BR1_2653987651
BR1_2600408638
BR1_2600373571
BR1_2599910191
BR1_2599226745
BR1_2599198937
BR1_2599021230
BR1_2598679980
BR1_2598666600
BR1_2598638392
BR1_2597488531
BR1_2597504050
BR1_2597084187
BR1_2596938130
BR1_2596750230
BR1_2596521403
BR1_2596449342
BR1_2596416779
BR1_2596414617
BR1_2594935729
BR1_2594971633
BR1_2594844071
BR1_2594841616
BR1_2594820026
BR1_2594609398
BR1_2594513971
BR1_2594349691
BR1_2594356075
BR1_2594343639
BR1_2594371727
BR1_2593969852
BR1_2593986163
BR1_2593973346
BR1_2593980046
BR1_2593896362
BR1_2593920498
BR1_2593866660
BR1_2593852981
BR1_2593697781
BR1_2593553822
BR1_2593580273
BR1_2593489985
BR1_2593527881
BR1_2592923837
BR1_2592899839
BR1_2592886556
BR1_2592846044
BR1_2592890629
BR1_2592447361
BR1_2592472420
BR1_2592368732
BR1_2592356247
BR1_259238

In [23]:
match_info = get_match_data(match_list[3], api_key)
match_info_df = pd.DataFrame(match_info)

game_dur, game_md, df_part = extract_match_info(match_info)

df_part

,championName,kills,death,assists,totalDamageToChampions,goldEarned,visioScore,role,teamId
0,Gnar,4,3,1,30214,19312,28,TOP,100
1,Lillia,1,7,6,25110,15328,37,JUNGLE,100
2,Yasuo,12,10,4,44392,20347,29,MIDDLE,100
3,Zeri,10,9,5,50128,21029,29,BOTTOM,100
4,Janna,0,10,14,8265,10773,72,UTILITY,100
5,Gangplank,5,6,6,17004,17193,20,TOP,200
6,Ivern,5,8,19,23585,16433,26,JUNGLE,200
7,Ryze,7,7,6,33867,20850,21,MIDDLE,200
8,Darius,19,3,13,44216,22313,15,BOTTOM,200
9,Lulu,3,3,27,14694,12214,87,UTILITY,200
